In [1]:
import polars as pl
from tqdm import tqdm

def create_link_prediction_dataset(df: pl.DataFrame, max_articles: int = None):
    """
    Create unbiased training dataset for link prediction.
    Input: Clean text without hyperlinks
    Target: Links that should exist
    """
    print("🎯 Creating unbiased link prediction dataset...")
    
    # Use subset if specified
    if max_articles:
        df = df.head(max_articles)
    
    training_data = []
    
    for row in tqdm(df.iter_rows(named=True), total=len(df), desc="Processing articles"):
        source_id = row["id"]
        source_title = row["title"]
        clean_text = row["text_withoutHref"]  # ✅ Clean text without links
        actual_links = row.get("links", []) or []  # ✅ Ground truth links
        
        # Extract target article IDs from links (ground truth)
        target_ids = []
        for link in actual_links:
            href_decoded = link.get("href_decoded", "")
            # Map href to article ID (you'd need a URL->ID mapping)
            target_id = map_href_to_article_id(href_decoded, df)
            if target_id and target_id != source_id:
                target_ids.append({
                    "target_id": target_id,
                    "anchor_text": link.get("anchor", ""),
                    "position": link.get("start_idx", 0)
                })
        
        training_data.append({
            "source_id": source_id,
            "source_title": source_title,
            "clean_text": clean_text,  # Input: text without links
            "target_links": target_ids,  # Labels: what links should exist
            "num_links": len(target_ids)
        })
    
    return pl.DataFrame(training_data)

def map_href_to_article_id(href_decoded: str, df: pl.DataFrame):
    """Map Wikipedia href to article ID in our dataset"""
    # Create title variations to match
    variations = [
        href_decoded,
        href_decoded.replace("_", " "),
        href_decoded.replace("%20", " "),
        href_decoded.replace("_", "%20")
    ]
    
    for variation in variations:
        matches = df.filter(pl.col("title").str.to_lowercase() == variation.lower())
        if matches.height > 0:
            return matches.select("id").to_series()[0]
    
    return None

In [2]:
def build_article_similarity_features(df: pl.DataFrame, client, collection_name: str):
    """
    Build features for link prediction using semantic similarity
    WITHOUT using existing hyperlinks
    """
    print("🧠 Building semantic similarity features...")
    
    features = []
    
    for row in tqdm(df.iter_rows(named=True), desc="Computing similarities"):
        source_id = row["id"]
        clean_text = row["text_withoutHref"]
        
        # Encode the clean text (no hyperlinks)
        try:
            # Try to get from Qdrant first
            similar_articles = client.recommend(
                collection_name=collection_name,
                positive=[int(source_id)],
                limit=50,  # Get top 50 similar articles
                with_payload=True
            )
        except:
            # Fallback: encode the clean text
            vec = encode_article_text(clean_text)
            similar_articles = client.search(
                collection_name=collection_name,
                query_vector=vec,
                limit=50
            )
        
        # Create candidate links based on similarity
        candidates = []
        for result in similar_articles:
            target_id = result.payload.get("id", result.id)
            if int(target_id) != int(source_id):
                candidates.append({
                    "target_id": target_id,
                    "similarity_score": result.score,
                    "target_title": result.payload.get("title", "")
                })
        
        features.append({
            "source_id": source_id,
            "candidates": candidates
        })
    
    return features

In [3]:
def create_training_samples(df: pl.DataFrame, similarity_features: list, negative_ratio: float = 2.0):
    """
    Create positive and negative training samples for link prediction
    """
    print("📊 Creating positive/negative training samples...")
    
    # Create mapping for fast lookup
    id_to_links = {}
    for row in df.iter_rows(named=True):
        source_id = row["id"]
        actual_links = row.get("links", []) or []
        target_ids = set()
        
        for link in actual_links:
            target_id = map_href_to_article_id(link.get("href_decoded", ""), df)
            if target_id:
                target_ids.add(target_id)
        
        id_to_links[source_id] = target_ids
    
    training_samples = []
    
    for feature_row in tqdm(similarity_features, desc="Creating samples"):
        source_id = feature_row["source_id"]
        candidates = feature_row["candidates"]
        actual_targets = id_to_links.get(source_id, set())
        
        positive_samples = []
        negative_samples = []
        
        for candidate in candidates:
            target_id = candidate["target_id"]
            similarity_score = candidate["similarity_score"]
            
            # Create feature vector
            features = {
                "source_id": source_id,
                "target_id": target_id,
                "similarity_score": similarity_score,
                "target_title": candidate["target_title"]
            }
            
            if target_id in actual_targets:
                # Positive sample - this link actually exists
                features["label"] = 1
                positive_samples.append(features)
            else:
                # Negative sample - this link doesn't exist
                features["label"] = 0
                negative_samples.append(features)
        
        # Balance positive/negative samples
        num_negatives = min(len(negative_samples), int(len(positive_samples) * negative_ratio))
        selected_negatives = negative_samples[:num_negatives]
        
        training_samples.extend(positive_samples)
        training_samples.extend(selected_negatives)
    
    return pl.DataFrame(training_samples)

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, precision_recall_curve
import numpy as np

def train_link_prediction_model(training_df: pl.DataFrame):
    """
    Train a simple link prediction model
    """
    print("🤖 Training link prediction model...")
    
    # Prepare features and labels
    X = training_df.select(["similarity_score"]).to_numpy()  # Start with just similarity
    y = training_df.select("label").to_numpy().flatten()
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Train model
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # Evaluate
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]
    
    print("📊 Model Performance:")
    print(classification_report(y_test, y_pred))
    
    return model, X_test, y_test, y_prob

def predict_links_for_article(model, client, article_id: int, df: pl.DataFrame, k: int = 10):
    """
    Predict top-k links for a given article using trained model
    """
    # Get article info
    article_row = df.filter(pl.col("id") == article_id)
    if article_row.height == 0:
        print(f"Article {article_id} not found")
        return
    
    article_title = article_row.select("title").to_series()[0]
    clean_text = article_row.select("text_withoutHref").to_series()[0]
    
    print(f"🔮 Predicting links for: '{article_title}' (ID: {article_id})")
    
    # Get similar articles (candidates)
    try:
        similar_articles = client.recommend(
            collection_name="wikipedia_fr",
            positive=[int(article_id)],
            limit=100,
            with_payload=True
        )
    except:
        vec = encode_article_text(clean_text)
        similar_articles = client.search(
            collection_name="wikipedia_fr",
            query_vector=vec,
            limit=100
        )
    
    # Predict for each candidate
    predictions = []
    for result in similar_articles:
        target_id = result.payload.get("id", result.id)
        if int(target_id) != int(article_id):
            # Create feature vector
            features = np.array([[result.score]])  # Just similarity for now
            
            # Predict probability
            prob = model.predict_proba(features)[0][1]
            
            predictions.append({
                "target_id": target_id,
                "target_title": result.payload.get("title", ""),
                "similarity_score": result.score,
                "link_probability": prob
            })
    
    # Sort by link probability and return top-k
    predictions.sort(key=lambda x: x["link_probability"], reverse=True)
    
    print(f"\n🎯 Top {k} predicted links:")
    for i, pred in enumerate(predictions[:k]):
        print(f"{i+1:2d}. '{pred['target_title']}' (ID: {pred['target_id']})")
        print(f"     Similarity: {pred['similarity_score']:.3f}, Link Prob: {pred['link_probability']:.3f}")
    
    return predictions[:k]

In [5]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams
from sentence_transformers import SentenceTransformer
import numpy as np
from pathlib import Path
import json
import pandas as pd
import torch
#If you already saved the model locally and are using docker
client = QdrantClient(host="localhost", port=6333, prefer_grpc=True)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer("models/multilingual-e5-large", device=device)

# Run docker container before
# docker run -d --name qdrant -p 6333:6333 -p 6334:6334 -v /home/Loris/EPFL/MA3/ML/project2/project2Rag/qdrant_storage:/qdrant/storage qdrant/qdrant:latest


In [6]:
import polars as pl
from tqdm import tqdm

df = pl.read_parquet("articles_fr_merged.parquet")


In [7]:
print(df.head())
print(df.shape)
print(df["links"][2])
print(df["title"][32])

shape: (5, 6)
┌─────┬───────────────────┬───────────────────┬───────────────────┬────────────┬───────────────────┐
│ id  ┆ title             ┆ text              ┆ links             ┆ link_count ┆ text_withoutHref  │
│ --- ┆ ---               ┆ ---               ┆ ---               ┆ ---        ┆ ---               │
│ i64 ┆ str               ┆ str               ┆ list[struct[5]]   ┆ u32        ┆ str               │
╞═════╪═══════════════════╪═══════════════════╪═══════════════════╪════════════╪═══════════════════╡
│ 3   ┆ Antoine Meillet   ┆ Antoine Meillet,  ┆ [{"https://fr.wik ┆ 65         ┆ Antoine Meillet,  │
│     ┆                   ┆ né le à &lt;a…    ┆ ipedia.org/wi…    ┆            ┆ né le à Mouli…    │
│ 7   ┆ Algèbre linéaire  ┆ L’algèbre         ┆ [{"https://fr.wik ┆ 111        ┆ L’algèbre         │
│     ┆                   ┆ linéaire est la   ┆ ipedia.org/wi…    ┆            ┆ linéaire est la   │
│     ┆                   ┆ bran…             ┆                   ┆          

In [8]:
# Step 1: Create unbiased training dataset
training_data = create_link_prediction_dataset(df.head(1000))

# Step 2: Build similarity features (using clean text only)
similarity_features = build_article_similarity_features(df.head(1000), client, "wikipedia_fr")

# Step 3: Create positive/negative samples
training_samples = create_training_samples(df.head(1000), similarity_features)

# Step 4: Train model
model, X_test, y_test, y_prob = train_link_prediction_model(training_samples)

# Step 5: Predict links for new articles
predictions = predict_links_for_article(model, client, 7, df, k=10)

🎯 Creating unbiased link prediction dataset...


Processing articles:   0%|          | 0/1000 [00:00<?, ?it/s]

Processing articles: 100%|██████████| 1000/1000 [07:33<00:00,  2.20it/s]


🧠 Building semantic similarity features...


Computing similarities: 0it [00:00, ?it/s]/tmp/ipykernel_11470/2553876889.py:17: DeprecationWarning: `recommend` method is deprecated and will be removed in the future. Use `query_points` instead.
  similar_articles = client.recommend(
Computing similarities: 1000it [01:30, 11.07it/s]


📊 Creating positive/negative training samples...


Creating samples: 100%|██████████| 1000/1000 [00:00<00:00, 70453.43it/s]


🤖 Training link prediction model...
📊 Model Performance:
              precision    recall  f1-score   support

           0       0.75      0.73      0.74       401
           1       0.55      0.57      0.56       229

    accuracy                           0.67       630
   macro avg       0.65      0.65      0.65       630
weighted avg       0.68      0.67      0.67       630

🔮 Predicting links for: 'Algèbre linéaire' (ID: 7)


/tmp/ipykernel_11470/1939682050.py:49: DeprecationWarning: `recommend` method is deprecated and will be removed in the future. Use `query_points` instead.
  similar_articles = client.recommend(



🎯 Top 10 predicted links:
 1. 'Parallélogramme' (ID: 8727)
     Similarity: 0.907, Link Prob: 1.000
 2. 'Matrice (mathématiques)' (ID: 15384)
     Similarity: 0.923, Link Prob: 0.980
 3. 'Géométrie' (ID: 1218)
     Similarity: 0.923, Link Prob: 0.970
 4. 'Conjugué' (ID: 32037)
     Similarity: 0.907, Link Prob: 0.950
 5. 'Idéal' (ID: 11001)
     Similarity: 0.910, Link Prob: 0.890
 6. 'Orbitale atomique' (ID: 35781)
     Similarity: 0.906, Link Prob: 0.880
 7. 'Ellipse (mathématiques)' (ID: 18697)
     Similarity: 0.904, Link Prob: 0.880
 8. 'Analyse réelle' (ID: 20035)
     Similarity: 0.912, Link Prob: 0.870
 9. 'Espace vectoriel' (ID: 1051)
     Similarity: 0.928, Link Prob: 0.820
10. 'Intégration (mathématiques)' (ID: 14294)
     Similarity: 0.919, Link Prob: 0.820


In [9]:
# TODO: Garder que les phrases avec liens hypertextes de chaque article, embedd ces phrases uniquement, et run cosine similarity entre les phrases elle mêmes et les articles en entier (ou les phrases hypertextes d'un autre article)

In [10]:
import polars as pl
from typing import Dict, List, Tuple
from sklearn.metrics import precision_score, recall_score, f1_score, average_precision_score
import numpy as np
from urllib.parse import quote

def create_url_to_id_mapping(df: pl.DataFrame) -> Dict[str, int]:
    """Create mapping from Wikipedia URLs/titles to article IDs"""
    url_to_id = {}
    
    for row in df.select(["id", "title"]).iter_rows(named=True):
        article_id = row["id"]
        title = row["title"]
        
        # Create URL-encoded version (like alg%C3%A8bre)
        encoded_title = quote(title.replace(" ", "_"), safe="")
        
        # Create various URL patterns that might appear in href_decoded
        patterns = [
            title,                                    # Original title
            title.replace(" ", "_"),                  # Spaces to underscores
            encoded_title,                           # URL encoded (alg%C3%A8bre)
            quote(title, safe=""),                   # Fully encoded
            title.replace(" ", "%20"),               # Spaces to %20
            title.lower(),                           # Lowercase
            title.lower().replace(" ", "_"),         # Lowercase with underscores
            encoded_title.lower()                    # Lowercase encoded
        ]
        
        for pattern in patterns:
            url_to_id[pattern] = article_id
    
    return url_to_id

def extract_true_links_for_article(df: pl.DataFrame, article_id: int, url_to_id: Dict[str, int]) -> List[int]:
    """Extract true target article IDs from the links array for a given article."""
    
    # Fixed Polars syntax
    article_row = df.filter(pl.col("id") == article_id)
    if article_row.height == 0:
        return []
    
    # Get links - handle potential None values properly
    links_series = article_row.select("links").to_series()
    if links_series.is_empty():  # Fixed: use is_empty() instead of direct boolean
        return []
    
    links = links_series[0]
    if links is None:  # Handle None case
        return []
    
    true_target_ids = []
    
    for link in links:
        href_decoded = link.get("href_decoded", "")
        
        # Try to map href_decoded to article ID
        target_id = None
        
        # Direct lookup first
        if href_decoded in url_to_id:
            target_id = url_to_id[href_decoded]
        else:
            # Try variations
            variations = [
                href_decoded.replace("_", " "),
                href_decoded.replace("%20", " "),
                href_decoded.replace("_", "%20"),
                quote(href_decoded.replace("_", " "), safe=""),  # Re-encode
                href_decoded.lower(),
                href_decoded.lower().replace("_", " "),
                href_decoded.lower().replace("%20", " ")
            ]
            
            for variation in variations:
                if variation in url_to_id:
                    target_id = url_to_id[variation]
                    break
        
        # Fixed: use proper comparison instead of 'and'
        if target_id is not None and target_id != article_id:
            true_target_ids.append(target_id)
    
    return list(set(true_target_ids))  # Remove duplicates

def validate_predictions_against_true_links(
    model,
    client,
    df: pl.DataFrame,
    collection_name: str = "wikipedia_fr",
    test_article_ids: List[int] = None,
    k_predictions: int = 20,
    similarity_threshold: float = 0.5
):
    """Validate link predictions against true hyperlinks from the dataset."""
    
    print("🔍 Validating predictions against true hyperlinks...")
    
    # Create URL to ID mapping
    url_to_id = create_url_to_id_mapping(df)
    print(f"📚 Created mapping for {len(url_to_id)} URL patterns")
    
    if test_article_ids is None:
        # Fixed: use proper Polars filtering
        articles_with_links = df.filter(pl.col("link_count") > 0)
        if not articles_with_links.is_empty():  # Fixed: use is_empty()
            test_article_ids = articles_with_links.select("id").to_series().to_list()[:100]
        else:
            test_article_ids = df.select("id").to_series().to_list()[:100]
    
    validation_results = []
    
    for article_id in tqdm(test_article_ids, desc="Validating articles"):
        try:
            # Get true links for this article
            true_links = extract_true_links_for_article(df, article_id, url_to_id)
            
            if not true_links:  # Fixed: direct boolean check is OK for lists
                continue
            
            # Get model predictions
            predicted_links = predict_links_for_article_validation(
                model, client, article_id, df, collection_name, k_predictions, similarity_threshold
            )
            
            if not predicted_links:  # Fixed: direct boolean check is OK for lists
                continue
            
            # Calculate metrics for this article
            metrics = calculate_article_metrics(true_links, predicted_links, k_predictions)
            
            validation_results.append({
                "article_id": article_id,
                "article_title": get_article_title(df, article_id),
                "num_true_links": len(true_links),
                "num_predictions": len(predicted_links),
                **metrics
            })
            
        except Exception as e:
            print(f"Error validating article {article_id}: {e}")
            continue
    
    # Calculate aggregate metrics
    aggregate_metrics = calculate_aggregate_metrics(validation_results)
    
    # Print results
    print_validation_results(validation_results, aggregate_metrics)
    
    return validation_results, aggregate_metrics

def get_article_title(df: pl.DataFrame, article_id: int) -> str:
    """Get article title by ID"""
    row = df.filter(pl.col("id") == article_id)
    if not row.is_empty():  # Fixed: use is_empty()
        return row.select("title").to_series()[0]
    return f"Unknown ID {article_id}"

def predict_links_for_article_validation(
    model, client, article_id: int, df: pl.DataFrame, 
    collection_name: str, k: int, similarity_threshold: float
) -> List[int]:
    """Get model predictions for validation"""
    
    # Get article info - Fixed Polars syntax
    article_row = df.filter(pl.col("id") == article_id)
    if article_row.is_empty():  # Fixed: use is_empty()
        return []
    
    clean_text = article_row.select("text_withoutHref").to_series()[0]
    
    # Get similar articles (candidates)
    try:
        similar_articles = client.recommend(
            collection_name=collection_name,
            positive=[int(article_id)],
            limit=k * 2,
            with_payload=True
        )
    except:
        # Fallback: encode the clean text
        vec = encode_article_text(clean_text)
        similar_articles = client.search(
            collection_name=collection_name,
            query_vector=vec,
            limit=k * 2
        )
    
    # Filter and predict
    predictions = []
    for result in similar_articles:
        target_id = result.payload.get("id", result.id)
        
        # Fixed: use proper comparison
        if (int(target_id) != int(article_id) and 
            result.score >= similarity_threshold):
            
            # Create feature vector for model
            features = np.array([[result.score]])
            
            try:
                prob = model.predict_proba(features)[0][1]
                predictions.append({
                    "target_id": target_id,
                    "similarity_score": result.score,
                    "link_probability": prob
                })
            except:
                continue
    
    # Sort by link probability and return top-k target IDs
    predictions.sort(key=lambda x: x["link_probability"], reverse=True)
    return [pred["target_id"] for pred in predictions[:k]]

# Rest of the functions remain the same...
def calculate_article_metrics(true_links: List[int], predicted_links: List[int], k: int) -> Dict:
    """Calculate precision, recall, F1 for a single article"""
    
    true_set = set(true_links)
    pred_set = set(predicted_links)
    
    tp = len(true_set.intersection(pred_set))
    fp = len(pred_set - true_set)
    fn = len(true_set - pred_set)
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    
    precision_at_k = tp / min(k, len(predicted_links)) if predicted_links else 0.0
    recall_at_k = tp / len(true_links) if true_links else 0.0
    
    return {
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "precision_at_k": precision_at_k,
        "recall_at_k": recall_at_k,
        "true_positives": tp,
        "false_positives": fp,
        "false_negatives": fn
    }

def calculate_aggregate_metrics(validation_results: List[Dict]) -> Dict:
    """Calculate aggregate metrics across all articles"""
    
    if not validation_results:
        return {}
    
    total_tp = sum(r["true_positives"] for r in validation_results)
    total_fp = sum(r["false_positives"] for r in validation_results)
    total_fn = sum(r["false_negatives"] for r in validation_results)
    
    micro_precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0.0
    micro_recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0.0
    micro_f1 = 2 * (micro_precision * micro_recall) / (micro_precision + micro_recall) if (micro_precision + micro_recall) > 0 else 0.0
    
    macro_precision = np.mean([r["precision"] for r in validation_results])
    macro_recall = np.mean([r["recall"] for r in validation_results])
    macro_f1 = np.mean([r["f1_score"] for r in validation_results])
    
    avg_precision_at_k = np.mean([r["precision_at_k"] for r in validation_results])
    avg_recall_at_k = np.mean([r["recall_at_k"] for r in validation_results])
    
    return {
        "num_articles": len(validation_results),
        "micro_precision": micro_precision,
        "micro_recall": micro_recall,
        "micro_f1": micro_f1,
        "macro_precision": macro_precision,
        "macro_recall": macro_recall,
        "macro_f1": macro_f1,
        "avg_precision_at_k": avg_precision_at_k,
        "avg_recall_at_k": avg_recall_at_k,
        "total_true_positives": total_tp,
        "total_false_positives": total_fp,
        "total_false_negatives": total_fn
    }

def print_validation_results(validation_results: List[Dict], aggregate_metrics: Dict):
    """Print detailed validation results"""
    
    print("\n" + "="*80)
    print("🎯 LINK PREDICTION VALIDATION RESULTS")
    print("="*80)
    
    print(f"\n📊 AGGREGATE METRICS ({aggregate_metrics['num_articles']} articles):")
    print(f"   Micro-averaged:")
    print(f"     Precision: {aggregate_metrics['micro_precision']:.3f}")
    print(f"     Recall:    {aggregate_metrics['micro_recall']:.3f}")
    print(f"     F1-Score:  {aggregate_metrics['micro_f1']:.3f}")
    
    print(f"\n   Macro-averaged:")
    print(f"     Precision: {aggregate_metrics['macro_precision']:.3f}")
    print(f"     Recall:    {aggregate_metrics['macro_recall']:.3f}")
    print(f"     F1-Score:  {aggregate_metrics['macro_f1']:.3f}")
    
    print(f"\n   Precision@K: {aggregate_metrics['avg_precision_at_k']:.3f}")
    print(f"   Recall@K:    {aggregate_metrics['avg_recall_at_k']:.3f}")

In [ ]:
# Complete pipeline - train model and validate
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Quick training pipeline + validation
def quick_link_prediction_pipeline(client, df, collection_name="wikipedia_fr", test_size=100):
    print("🚀 Quick Link Prediction Pipeline")
    
    # Step 1: Create simple training data from similarity scores
    print("📚 Creating training data...")
    training_samples = []
    url_to_id = create_url_to_id_mapping(df.head(500))  # Use subset for speed
    
    for row in df.head(200).iter_rows(named=True):  # Small training set
        source_id = row["id"]
        true_links = extract_true_links_for_article(df, source_id, url_to_id)
        
        if not true_links:
            continue
            
        try:
            # Get similar articles
            similar = client.recommend(collection_name=collection_name, positive=[int(source_id)], limit=20, with_payload=True)
            
            for result in similar:
                target_id = result.payload.get("id", result.id)
                if int(target_id) != int(source_id):
                    label = 1 if target_id in true_links else 0
                    training_samples.append([result.score, label])
        except:
            continue
    
    if len(training_samples) < 10:
        print("❌ Not enough training samples")
        return None, None
    
    # Step 2: Train simple model
    print(f"🤖 Training model on {len(training_samples)} samples...")
    X = np.array([s[0] for s in training_samples]).reshape(-1, 1)
    y = np.array([s[1] for s in training_samples])
    
    model = RandomForestClassifier(n_estimators=50, random_state=42)
    model.fit(X, y)
    
    # Step 3: Validate
    print("🔍 Running validation...")
    validation_results, metrics = validate_predictions_against_true_links(
        model=model,
        client=client,
        df=df,
        collection_name=collection_name,
        test_article_ids=None,
        k_predictions=10,
        similarity_threshold=0.3
    )
    
    return validation_results, metrics

# ONE-LINER USAGE:
validation_results, metrics = quick_link_prediction_pipeline(client, df)

🚀 Quick Link Prediction Pipeline
📚 Creating training data...


/tmp/ipykernel_11470/4097986851.py:23: DeprecationWarning: `recommend` method is deprecated and will be removed in the future. Use `query_points` instead.
  similar = client.recommend(collection_name=collection_name, positive=[int(source_id)], limit=20, with_payload=True)


🤖 Training model on 2500 samples...
🔍 Running validation...
🔍 Validating predictions against true hyperlinks...
📚 Created mapping for 25059409 URL patterns


Validating articles:   0%|          | 0/100 [00:00<?, ?it/s]/tmp/ipykernel_11470/3086961291.py:174: DeprecationWarning: `recommend` method is deprecated and will be removed in the future. Use `query_points` instead.
  similar_articles = client.recommend(
Validating articles:   1%|          | 1/100 [00:01<01:44,  1.06s/it]/tmp/ipykernel_11470/3086961291.py:174: DeprecationWarning: `recommend` method is deprecated and will be removed in the future. Use `query_points` instead.
  similar_articles = client.recommend(
Validating articles:   2%|▏         | 2/100 [00:01<01:19,  1.23it/s]/tmp/ipykernel_11470/3086961291.py:174: DeprecationWarning: `recommend` method is deprecated and will be removed in the future. Use `query_points` instead.
  similar_articles = client.recommend(
Validating articles:   3%|▎         | 3/100 [00:02<01:10,  1.38it/s]/tmp/ipykernel_11470/3086961291.py:174: DeprecationWarning: `recommend` method is deprecated and will be removed in the future. Use `query_points` inst


🎯 LINK PREDICTION VALIDATION RESULTS

📊 AGGREGATE METRICS (100 articles):
   Micro-averaged:
     Precision: 0.270
     Recall:    0.017
     F1-Score:  0.033

   Macro-averaged:
     Precision: 0.270
     Recall:    0.049
     F1-Score:  0.055

   Precision@K: 0.270
   Recall@K:    0.049


: 